RAG:

- Retrieval → Abruf von externen Informationen (z. B. aus einer Datenbank, Dokumenten oder dem Web).
- Augmented → Diese Informationen werden in den Generierungsprozess „eingespeist“ und erweitern das Modellwissen.
- Generation → Das Sprachmodell erzeugt darauf basierend eine Antwort oder ein neues Dokument.

In zweiten Schritt wird Augmented, also das Erzeugen von Chunks ausgeführt

Man braucht dafür zumindest die Lib: langchain-text-splitters

In [ ]:
## load pdf 
from langchain_community.document_loaders import PyPDFLoader

pdfLoader = PyPDFLoader('../1-DataIgnestion/Tempel.pdf')
pdfFile = pdfLoader.load()
print(pdfFile[0])



page_content='Neunter Awa, 3929 (68 unseres Zeitalters) Römer verbrennen Zweiten Tempel. Ein Drittel 
der Bevölkerung wurde ermordet oder als Sklaven verkauft. 
Johanan Ben Sakai: Oberkauft von Waisen nach der Zerstörung vom 2. Temepl 
Anführen der Schamai-Schule. Sohar schreibt, dass er Sod kannte. 
Nach der Zerstörung vom 2. Tempel gründete er Jeschiwa in Jawne. 
 
Joschua ben Naija, Schüler von Johanan Ben Zakai. Er hat Unkelus ausgebildet. 
 
Eliezer ben Orkanos, Schüler von Johanan ben Zaki. Sein Schüler war Unkelus und Rabi 
Akiwa. Er war nah zu der Schamai-Schule. Er wurde von der Gemeinde entfernt, weil er 
sich nicht nach der Mehrheit richtete. Nach dem Tod von Elizer ben Orkanos wurde er zu 
der Gemeinde wieder hinzugefügt und seine Gesetze werden angewandt. Er hat das 
Buch „Pirkei deabi Elizer“ geschrieben. 
Raban Gamliel: Verwandter vom König David. Er hat „Amida“ und „Pessah Seder „ 
herausgegeben. Er war Oberhaupt vom Sandhedrin. 
 
Hania ben Dosa: Hat viele Wunder vollb

In [ ]:
## splitte 'Documents' rekursiv Buchstabe für Buchstabe

from langchain_text_splitters import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=50, chunk_overlap=5)
## hier hat man schon den Text als "Document"
print(type(pdfFile[0]))
final_documents = text_splitter.split_documents(pdfFile) ##splitte Documents
final_documents[0].page_content

<class 'langchain_core.documents.base.Document'>


'Neunter Awa, 3929 (68 unseres Zeitalters) Römer'

In [ ]:
## Nehme Text und erzeuge Chunks von Typ 'Document'ArithmeticError

from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

with open("../1-DataIgnestion/HKurs.txt") as text:
    content = text.read() ## hier hast man nur text, keinen 'Document' Type

recursive_text_splitter = RecursiveCharacterTextSplitter(chunk_size=20, chunk_overlap=10)
documents = recursive_text_splitter.create_documents([content])
print(type(documents[0]))
print(documents[0]) ##man sieht eine Überlappung hier 
print(documents[1]) ##und hier 
print(documents[2])


<class 'langchain_core.documents.base.Document'>
page_content='Beispiel: Alltag am'
page_content='am Donnerstag'
page_content='(A2-Niveau,'


In [22]:
## man kann auch mit einem einfachen CharacterTextSplitter arbeiten:

from langchain_text_splitters import CharacterTextSplitter

text_splitter = CharacterTextSplitter("\n\n", chunk_size=20, chunk_overlap=4)
documents = text_splitter.create_documents([content])
## dieser Splitter sagt z.b Created a chunk of size 604, which is longer than the specified 20
## weil er den Seperator "\n\n" nicht findet
print(type(documents[0]))
print(f"INHALT_0: \n {documents[0]}") ##man sieht eine Überlappung hier 
print(f"INHALT_1: \n {documents[1]}") ##und hier 
print(f"INHALT_2: \n {documents[2]}")


Created a chunk of size 604, which is longer than the specified 20
Created a chunk of size 360, which is longer than the specified 20
Created a chunk of size 146, which is longer than the specified 20
Created a chunk of size 181, which is longer than the specified 20
Created a chunk of size 97, which is longer than the specified 20
Created a chunk of size 184, which is longer than the specified 20
Created a chunk of size 151, which is longer than the specified 20
Created a chunk of size 138, which is longer than the specified 20
Created a chunk of size 85, which is longer than the specified 20
Created a chunk of size 69, which is longer than the specified 20


<class 'langchain_core.documents.base.Document'>
INHALT_0: 
 page_content='Beispiel: Alltag am Donnerstag (A2-Niveau, männliche Person)
Morgens
- Am Donnerstag stehe ich um sieben Uhr auf.
- Ich wasche mich und ziehe mich an.
- Dann frühstücke ich: Brot mit Käse und ein Glas Saft.
- Um acht Uhr gehe ich zur Arbeit / zur Uni.
Mittags
- Mittags esse ich in der Kantine oder zu Hause.
- Ich mag Suppe und Salat.
- Nach dem Essen mache ich eine kleine Pause.
- Um ein Uhr arbeite ich weiter.
Abends
- Am Abend komme ich um sechs Uhr nach Hause.
- Ich koche das Abendessen, zum Beispiel Nudeln mit Gemüse.
- Danach sehe ich fern oder lese ein Buch.
- Um zehn Uhr gehe ich ins Bett.'
INHALT_1: 
 page_content='Übungsidee
- Versuche, die Sätze laut nachzusprechen.
- Danach kannst du kleine Änderungen machen: z. B. „Ich stehe um acht Uhr auf“ oder „Ich esse Pizza am Abend“.
- So übst du, über deinen eigenen Alltag zu sprechen.
Willst du, dass ich dir gleich ein Mini-Rollenspiel vorbereite, wo du mir d

In [ ]:
## dieser Splitter unterstütz nur h1,h2,h3 usw... tags

from langchain_text_splitters import HTMLHeaderTextSplitter

html_string = """
    <!DOCTYPE html>
<html lang="de">
<head>
    <meta charset="UTF-8" />
    <meta name="viewport" content="width=device-width, initial-scale=1.0" />
    <title>Beispiel HTML Dokument</title>
</head>
<body>
    <header>
        <h1>Willkommen auf meiner Beispielseite</h1>
        <nav>
            <ul>
                <li><a href="#section1">Abschnitt 1</a></li>
                <li><a href="#section2">Abschnitt 2</a></li>
            </ul>
        </nav>
    </header>

    <main>
        <section id="section1">
            <h2>Abschnitt 1</h2>
            <div>
                <p>Dies ist ein Beispieltext in einem <strong>Paragraph</strong>. <br />
                Hier wird ein Zeilenumbruch demonstriert.</p>
            </div>
        </section>

        <section id="section2">
            <h2>Abschnitt 2</h2>
            <article>
                <p>Ein weiterer Absatz mit einem <em>kursiven</em> Text und einem Link: 
                <a href="https://example.com">Beispielseite</a>.</p>
                <ul>
                    <li>Listenpunkt 1</li>
                    <li>Listenpunkt 2</li>
                </ul>
            </article>
        </section>
    </main>

    <footer>
        <small>&copy; 2025 Beispielseite</small>
    </footer>
</body>
</html>
"""
## list with tuple
headers_to_split_on = [
    ("h1", "header1"),
    ("h2", "header2"),
    ("h3", "header3")
]

html_splitter = HTMLHeaderTextSplitter(headers_to_split_on)
splitted_html_text = html_splitter.split_text(html_string)

print(type(splitted_html_text[0])) ## In der Liste sind die benötigten Dokumente
print(splitted_html_text[0].page_content)
print(splitted_html_text[1].page_content)

html_splitter.split_text_from_url("www.zdf.de") ##man kann auch eine url übergeben und dessen html nach h-tags splitten

<class 'langchain_core.documents.base.Document'>
Willkommen auf meiner Beispielseite
Abschnitt 1  
Abschnitt 2


In [ ]:
## Dieser Splitter unterstützt beliebige Tags

from langchain_text_splitters import HTMLSectionSplitter

html_without_h_tags="""

<!doctype html>
<html>
  <body>
    <div class="section">
      <p>Dies ist Abschnitt 1 mit etwas Text.</p>
    </div>

    <div class="section">
      <p>Abschnitt 2 enthält eine Liste:</p>
      <ul>
        <li>Eintrag A</li>
        <li>Eintrag B</li>
      </ul>
    </div>

    <div class="footer">
      <p>Footer-Bereich ohne Überschriften.</p>
    </div>
  </body>
</html>
"""

## man muss zwingend mind. einen h-tag übergeben 
## aber wenn man einen html-dokument ohne h-tags hat, wird es nach dem nächsten splitter
## gechunkt, und das ist der div-tag in diesem Beispiel
headers_to_split_on = [
    ("h1", "header1"),
    ("div", "header2")
]

html_splitter = HTMLSectionSplitter(headers_to_split_on)
chunked_html_text = html_splitter.split_text(html_without_h_tags)
print(type(chunked_html_text[0])) ## In der Liste sind die benötigten Dokumente
for document in chunked_html_text:

    print(f"div-splitter: {document.metadata}")
    print(document.page_content)


<class 'langchain_core.documents.base.Document'>
div-splitter: {'header2': 'Dies ist Abschnitt 1 mit etwas Text.'}
Dies ist Abschnitt 1 mit etwas Text.
div-splitter: {'header2': 'Abschnitt 2 enthält eine Liste:\n\nEintrag A\nEintrag B'}
Abschnitt 2 enthält eine Liste: 
 
 Eintrag A 
 Eintrag B
div-splitter: {'header2': 'Footer-Bereich ohne Überschriften.'}
Footer-Bereich ohne Überschriften.


In [ ]:
## Splitte JSON

import json
import requests
from langchain_text_splitters import RecursiveJsonSplitter


json_data = requests.get("https://api.smith.langchain.com/openapi.json").json()
json_splitter = RecursiveJsonSplitter(max_chunk_size = 300)
json_chunks = json_splitter.split_json(json_data)

for chunk in json_chunks[:3]:
    print(chunk)

## create Dokuments from json

chunked_docs = RecursiveJsonSplitter.create_documents(texts=[json_data])